In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
plt.rc('font', family='Malgun Gothic')

import datasets

In [ ]:
df_lesson_page = datasets.enter.lesson_page()
df_complete_lesson = datasets.complete.lesson()
df_click_lesson_question = datasets.click.lesson_page_related_question_box()

In [ ]:
# 로그 시간 date time 형식으로 변환
df_lesson_page['client_event_time'] = pd.to_datetime(df_lesson_page['client_event_time'])
df_complete_lesson['client_event_time'] = pd.to_datetime(df_complete_lesson['client_event_time'])
df_click_lesson_question['client_event_time'] = pd.to_datetime(df_click_lesson_question['client_event_time'])

In [ ]:
# # 중복 제거
# df_lesson_page.drop_duplicates(inplace=True)
# df_complete_lesson.drop_duplicates(inplace=True)
# df_click_lesson_question.drop_duplicates(inplace=True)

# # 컬럼 .  _ 으로 변경
# df_lesson_page.columns = df_lesson_page.columns.str.replace('.', '_')
# df_complete_lesson.columns = df_complete_lesson.columns.str.replace('.', '_')
# df_click_lesson_question.columns = df_click_lesson_question.columns.str.replace('.', '_')

# # 날짜 정리
# df_lesson_page = df_lesson_page[df_lesson_page['client_event_time'].dt.year.isin([2022, 2023])]
# df_complete_lesson = df_complete_lesson[df_complete_lesson['client_event_time'].dt.year.isin([2022, 2023])]
# df_click_lesson_question = df_click_lesson_question[df_click_lesson_question['client_event_time'].dt.year.isin([2022, 2023])]

# # 기초 처리 저장
# df_lesson_page.to_csv('./data/enter.lesson_page.csv',encoding='UTF-8')
# df_complete_lesson.to_csv('./data/complete.lesson.csv',encoding='UTF-8')
# df_click_lesson_question.to_csv('./data/click.lesson_page_related_question_box.csv',encoding='UTF-8')

In [ ]:
# 데이터 시간 범위 확인
print("df_lesson_page:")
print("Min:", df_lesson_page['client_event_time'].min())
print("Max:", df_lesson_page['client_event_time'].max())

print("\ndf_complete_lesson:")
print("Min:", df_complete_lesson['client_event_time'].min())
print("Max:", df_complete_lesson['client_event_time'].max())

print("\ndf_click_lesson_question:")
print("Min:", df_click_lesson_question['client_event_time'].min())
print("Max:", df_click_lesson_question['client_event_time'].max())

In [ ]:
df_lesson_page.head(4)

In [ ]:
df_lesson_page

In [ ]:
# 데이터 중복 검정
df_complete_lesson.loc[df_complete_lesson.duplicated(subset=['user_id','lesson_id'])]

In [ ]:
# 데이터 중복 해결
df_complete_lesson = df_complete_lesson.drop_duplicates(subset=['user_id','lesson_id'],keep='first')

In [ ]:
df_click_lesson_question.isna().sum()

In [ ]:
df_click_lesson_question.shape

In [ ]:
# 월별 이용자수 추이 확인
monthly_users_enter_lesson = df_lesson_page.resample('ME', on='client_event_time')['user_id'].count()
# 인덱스를 month로, %Y-%m 형태로 변환
monthly_users_enter_lesson.index = monthly_users_enter_lesson.index.strftime('%Y-%m')
monthly_users_enter_lesson.index.name = 'month'
# 결과 출력
print(monthly_users_enter_lesson)

In [ ]:
# 월별 이용자수 추이 확인
monthly_users_complete_lesson = df_complete_lesson.resample('ME', on='client_event_time')['user_id'].count()
# 인덱스를 month로, %Y-%m 형태로 변환
monthly_users_complete_lesson.index = monthly_users_complete_lesson.index.strftime('%Y-%m')
monthly_users_complete_lesson.index.name = 'month'
# 결과 출력
print(monthly_users_complete_lesson)

In [ ]:
# 월별 이용자수 추이 확인
monthly_users_lesson_question = df_click_lesson_question.resample('ME', on='client_event_time')['user_id'].count()
# 인덱스를 month로, %Y-%m 형태로 변환
monthly_users_lesson_question.index = monthly_users_lesson_question.index.strftime('%Y-%m')
monthly_users_lesson_question.index.name = 'month'

# 결과 출력
print(monthly_users_lesson_question)

In [ ]:
# 컬럼명 수정
monthly_users_enter_lesson = monthly_users_enter_lesson.reset_index().rename(columns={'user_id':'enter_count'})
monthly_users_complete_lesson = monthly_users_complete_lesson.reset_index().rename(columns={'user_id':'complete_count'})
monthly_users_lesson_question = monthly_users_lesson_question.reset_index().rename(columns={'user_id':'question_count'})


In [ ]:
# 데이터프레임 병합
merged_df = pd.merge(monthly_users_enter_lesson, monthly_users_complete_lesson, on='month')
merged_df = pd.merge(merged_df, monthly_users_lesson_question, on='month')

# 결과 출력
display(merged_df)

In [ ]:
# 플롯그리기
fig, ax = plt.subplots(figsize=(12, 6))

# 바 넓이 설정
bar_width = 0.35

# Set the positions of the bars on the x-axis
r1 = range(len(merged_df['month']))
r2 = [x + bar_width for x in r1]

# 바 그리기
ax.bar(r1, merged_df['enter_count'], color='royalblue', width=bar_width, edgecolor='black', label='Enter Count')
ax.bar(r2, merged_df['complete_count'], color='orange', width=bar_width, edgecolor='black', label='Complete Count')

# 라벨 붙이기
ax.set_xlabel('Month', fontweight='bold')
ax.set_ylabel('Count', fontweight='bold')
ax.set_title('Monthly Lesson Enter and Complete Counts')
ax.set_xticks([r + bar_width/2 for r in range(len(merged_df['month']))])
ax.set_xticklabels(merged_df['month'], rotation=45)

# 레전드 붙이기
ax.legend()

# 그래프 그리기
plt.show()

In [ ]:
# 할당 해제
del monthly_users_enter_lesson
del monthly_users_lesson_question
del monthly_users_complete_lesson

In [ ]:
# lesson_id별로 count 계산
lesson_page_counts = df_lesson_page.groupby('lesson_id')['user_id'].size().reset_index(name='enter_count')
complete_lesson_counts = df_complete_lesson.groupby('lesson_id')['user_id'].size().reset_index(name='complete_count')
click_lesson_question_counts = df_click_lesson_question.groupby('lesson_id')['user_id'].size().reset_index(name='click_question_count')

merged_df = pd.merge(lesson_page_counts,complete_lesson_counts, on='lesson_id',how='outer')
merged_df = pd.merge(merged_df, click_lesson_question_counts, on='lesson_id',how='outer')


In [ ]:
merged_df.shape

In [ ]:
merged_df = merged_df.fillna(0)

In [ ]:
# 0으로 나누어지는 케이스를 분기시켜 처리.
# merged_df['complete/enter'] = np.where(
#     merged_df['enter_count'] == 0, 999,
#     round(merged_df['complete_count'] / merged_df['enter_count'] * 100, 2)
# )

# 분자와 분모에 1씩 더해서 나누기.
merged_df['complete/enter'] = round((merged_df['complete_count']+1) / (merged_df['enter_count']+1) * 100, 2)

In [ ]:
merged_df.loc[merged_df['complete/enter'] > 100].sort_values(by='complete/enter')

In [ ]:
# complete/enter를 0~10, 10~20, ..., 100 이상으로 분할
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, float('inf')]
labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100+']
merged_df['complete/enter_group'] = pd.cut(merged_df['complete/enter'], bins=bins, labels=labels, right=True)

# 각 그룹에 몇 개의 행들이 포함되어 있는지 count
group_counts = merged_df['complete/enter_group'].value_counts().sort_index()

# 결과 출력
print(group_counts)

In [ ]:
# 바 그래프 그리기
sns.barplot(x=group_counts.index, y=group_counts.values, hue=group_counts.index, legend=False, palette='Blues_d', edgecolor='black')

plt.title("Lesson 별 Enter 대비 Complete 비율")
plt.ylabel("Counts")
plt.xlabel("초과-이하 (%)")
# 그래프 그리기
plt.show()

In [ ]:
merged_df.groupby('complete/enter_group')['click question_count'].mean()

In [ ]:
# 그룹별 click question_count의 평균 계산
group_mean_click_question = merged_df.groupby('complete/enter_group')['click question_count'].mean().reset_index()

# 바 그래프 그리기
sns.barplot(x='complete/enter_group', y='click question_count', data=group_mean_click_question, palette='Blues_d', hue='complete/enter_group', legend=False, edgecolor='black')

plt.title("완강율 그룹별 평균 질문 클릭수")
plt.ylabel("평균 Question Counts")
plt.xlabel("완강율(%) 그룹")
# 그래프 그리기
plt.show()

In [ ]:
# Enter 대비 질문수 백분율 계산
merged_df['click/enter'] = round((merged_df['click question_count'] / merged_df['enter_count']) * 100, 2)

# Complete 대비 질문수 백분율 계산
merged_df['click/complete'] = round((merged_df['click question_count'] / merged_df['complete_count']) * 100, 2)

# 그룹별 평균 계산
group_mean_click_enter = merged_df.groupby('complete/enter_group')['click/enter'].mean().reset_index()
group_mean_click_complete = merged_df.groupby('complete/enter_group')['click/complete'].mean().reset_index()

# 그래프 그리기
fig, ax = plt.subplots(2, 1, figsize=(8, 12))

# Enter 대비 질문수 백분율 바 그래프
sns.barplot(x='complete/enter_group', y='click/enter', data=group_mean_click_enter, palette='Blues_d', hue='complete/enter_group', ax=ax[0], edgecolor='black')
ax[0].set_title("완강율 그룹별 Enter 대비 질문클릭율")
ax[0].set_ylabel("Enter 대비 질문클릭율 (%)")
ax[0].set_xlabel("완강율(%) 그룹")

# Complete 대비 질문수 백분율 바 그래프
sns.barplot(x='complete/enter_group', y='click/complete', data=group_mean_click_complete, palette='Blues_d', hue='complete/enter_group', ax=ax[1], edgecolor='black')
ax[1].set_title("완강율 그룹별 Complete 대비 질문클릭율")
ax[1].set_ylabel("Complete 대비 질문클릭율 (%)")
ax[1].set_xlabel("완강율(%) 그룹")

# 그래프 그리기
plt.tight_layout()
plt.show()

In [ ]:
start_content = datasets.start.content()
start_content.columns = start_content.columns.str.replace('.', '_')

In [ ]:
start_content.head(4)

In [ ]:
start_content = start_content[['content_id','content_difficulty']]

In [ ]:
del merged_df

In [ ]:
# complete_lesson에서 content_id 컬럼을 가져와 merged_df와 lesson_id 기준으로 병합
merged_df = pd.merge(merged_df, df_complete_lesson[['lesson_id', 'content_id']], on='lesson_id', how='left')

# start_content 데이터프레임을 content_id 기준으로 병합
merged_df = pd.merge(merged_df, start_content, on='content_id', how='left')

# 결과 출력
display(merged_df)